<a href="https://colab.research.google.com/github/AnthonyM99/Seminario-Profesional/blob/master/Lab5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%tensorflow_version 2.x

import tensorflow as tf
import tensorflow_hub as hub

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, Flatten, GlobalAveragePooling1D, Bidirectional, LSTM

import numpy as np
import pandas as pd
import matplotlib.pylab as plt
import os, re, json, functools

#dark mode
plt.rc_context({'xtick.color':'w', 'ytick.color':'w', 'text.color':'w', 'axes.labelcolor':'w'})

seed=1234
np.random.seed(seed)
tf.random.set_seed(seed)

TensorFlow 2.x selected.


In [2]:
pip install kaggle --upgrade

Requirement already up-to-date: kaggle in /usr/local/lib/python3.6/dist-packages (1.5.6)


In [3]:
os.environ['KAGGLE_USERNAME'] = "anthonymenendez"
os.environ['KAGGLE_KEY'] = "37d0f4f25110139ce59a572a2c7096b9"
!kaggle datasets download septa97/100k-courseras-course-reviews-dataset
!unzip -o '100k-courseras-course-reviews-dataset.zip'

100k-courseras-course-reviews-dataset.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  100k-courseras-course-reviews-dataset.zip
  inflating: reviews.csv             
  inflating: reviews_by_course.csv   


In [4]:
os.listdir()

['.config',
 'reviews.csv',
 '100k-courseras-course-reviews-dataset.zip',
 'reviews_by_course.csv',
 'sample_data']

In [5]:
Reviews = pd.read_csv('reviews_by_course.csv')
Reviews.head

<bound method NDFrame.head of           CourseId                                             Review  Label
0       2-speed-it                                             BOring      1
1       2-speed-it                                            Bravo !      5
2       2-speed-it                                           Very goo      5
3       2-speed-it  Great course - I recommend it for all, especia...      5
4       2-speed-it    One of the most useful course on IT Management!      5
...            ...                                                ...    ...
140315        zika  Trendy topic with talks from expertises in the...      4
140316        zika  Wonderful! Simple and clear language, good ins...      5
140317        zika   an interesting and fun course. thanks. dr quincy      5
140318        zika  very broad perspective, up to date information...      4
140319        zika  An informative course on the social and financ...      4

[140320 rows x 3 columns]>

In [6]:
Reviews.columns

Index(['CourseId', 'Review', 'Label'], dtype='object')

In [0]:
Reviews = Reviews[Reviews['CourseId'] != '0']
Reviews = Reviews[~pd.isna(Reviews['Review'])]

In [8]:
dataset = Reviews[['Review', 'Label']] 
dataset

,Review,Label
0,BOring,1
1,Bravo !,5
2,Very goo,5
3,"Great course - I recommend it for all, especia...",5
4,One of the most useful course on IT Management!,5
...,...,...
140315,Trendy topic with talks from expertises in the...,4
140316,"Wonderful! Simple and clear language, good ins...",5
140317,an interesting and fun course. thanks. dr quincy,5
140318,"very broad perspective, up to date information...",4


In [0]:
def preprocess_text(sen):
    sentence = re.sub('[^a-zA-Z]', ' ', sen)
    sentence = re.sub(r"\s+[a-zA-Z]\s+", ' ', sentence)
    sentence = re.sub(r'\s+', ' ', sentence)

    return sentence

In [10]:
train_split_processed = dataset['Review'].apply(lambda x: preprocess_text(x))
train_split_processed

0                                                    BOring
1                                                    Bravo 
2                                                  Very goo
3         Great course recommend it for all especially I...
4           One of the most useful course on IT Management 
                                ...                        
140315    Trendy topic with talks from expertises in the...
140316    Wonderful Simple and clear language good instr...
140317       an interesting and fun course thanks dr quincy
140318    very broad perspective up to date information ...
140319    An informative course on the social and financ...
Name: Review, Length: 140317, dtype: object

In [11]:
dataset[dataset['Label'] != 3]
dataset['Positivity'] = np.where(dataset['Label'] > 3, 1, 0)
cols = [ 'Label']
dataset.drop(cols, axis=1, inplace=True)
dataset.head()

,Review,Positivity
0,BOring,0
1,Bravo !,1
2,Very goo,1
3,"Great course - I recommend it for all, especia...",1
4,One of the most useful course on IT Management!,1


In [12]:
cols = [ 'Review']
dataset.drop(cols, axis=1, inplace=True)
dataset['Review'] = train_split_processed
dataset.head()

,Positivity,Review
0,0,BOring
1,1,Bravo
2,1,Very goo
3,1,Great course recommend it for all especially I...
4,1,One of the most useful course on IT Management


In [14]:
dataset

,Positivity,Review
0,0,BOring
1,1,Bravo
2,1,Very goo
3,1,Great course recommend it for all especially I...
4,1,One of the most useful course on IT Management
...,...,...
140315,1,Trendy topic with talks from expertises in the...
140316,1,Wonderful Simple and clear language good instr...
140317,1,an interesting and fun course thanks dr quincy
140318,1,very broad perspective up to date information ...


In [13]:
dataset.shape

(140317, 2)

In [15]:
train_split = dataset.sample(frac = 0.7,random_state=42)
test_split = dataset.drop(train_split.index).sample(frac = 0.5)
validation_split = dataset.drop(train_split.index).drop(test_split.index)
train_split.shape, test_split.shape, validation_split.shape

((98222, 2), (21048, 2), (21047, 2))

In [15]:
train_split.dtypes

Positivity     int64
Review        object
dtype: object

In [18]:
max_len = train_split['Review'].map(lambda x: len(x)).max()
max_len

7471

In [24]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(train_split['Review'])
train_sequences = tokenizer.texts_to_sequences(train_split['Review'])
train_data = pad_sequences(train_sequences, maxlen=max_len, padding='pre')

test_sequences = tokenizer.texts_to_sequences(test_split['Review'])
test_data  = pad_sequences(test_sequences, maxlen=max_len, padding='pre')

validation_sequences = tokenizer.texts_to_sequences(validation_split['Review'])
validation_data  = pad_sequences(validation_sequences, maxlen=max_len, padding='pre')

train_data.shape, test_data.shape, validation_data.shape

((98222, 7471), (21048, 7471), (21047, 7471))

In [27]:
train_data.dtype, test_data.dtype, validation_data.dtype

(dtype('int32'), dtype('int32'), dtype('int32'))

In [28]:
train_labels = train_split['Positivity']
test_labels = test_split['Positivity']
validation_labels = validation_split['Positivity']
train_labels.shape, test_labels.shape, validation_labels.shape

((98222,), (21048,), (21047,))

In [29]:
train_labels.dtype, test_labels.dtype, validation_labels.dtype

(dtype('int64'), dtype('int64'), dtype('int64'))

In [30]:
train_Label = pd.get_dummies(train_labels).values
test_Label = pd.get_dummies(test_labels).values
validation_Label = pd.get_dummies(validation_labels).values

train_Label.shape,test_Label.shape,validation_Label.shape

((98222, 2), (21048, 2), (21047, 2))

In [31]:
print(train_data.shape,train_Label.shape)
print(test_data.shape,test_Label.shape)

(98222, 7471) (98222, 2)
(21048, 7471) (21048, 2)


In [33]:
model = Sequential([
    Embedding(98222, 32, input_length = 7471),
    Bidirectional(LSTM(32)),
    Dense(2, activation='sigmoid')
])

model.summary()


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 7471, 32)          3143104   
_________________________________________________________________
bidirectional_1 (Bidirection (None, 64)                16640     
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 130       
Total params: 3,159,874
Trainable params: 3,159,874
Non-trainable params: 0
_________________________________________________________________


In [0]:
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [0]:
history = model.fit(train_data, train_Label, epochs=4,
                      steps_per_epoch=2
                      )

Train on 98222 samples
Epoch 1/4


In [0]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.ylabel('Accuracy')
plt.ylim([min(plt.ylim()),1])
plt.title('Training and Validation Accuracy')

plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.ylabel('Cross Entropy')
plt.ylim([0,0.8])
plt.title('Training and Validation Loss')
plt.xlabel('epoch')
plt.show()